In [1]:
import numpy as np
from qutip import *
from helpers.estimation import *
from helpers.data_processing import *
from helpers.targets import *
import exp_params
import scipy as sc

USING W-FUNCTION POINTS


### Check yourself: print the function, dimension, and number of displacements

In [2]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

W
D=6
nD=35


### State list 

In [3]:
state_list = exp_params.cat_state_list
print(state_list)

['cat-eve-1' 'cat-nmp-1' 'cat-nop-1' 'cat-odd-1']


### Choose whether to use grape or ideal for state preparation and target states

In [4]:
use_grape = True

# GRAPE generated target states
target_states_directory = r"target_states" # path to directory of target states generated from grape

### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [5]:
# Load inverse map variables for converting pe to rho
map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_W.npz"
)
W = map_variables["W"]
beta = map_variables["beta"]


### Function for protocol simulation with QuTiP

In [6]:
def wigner_sim(rho_init, c_ops_qc):
    H_dis = exp_params.H0

    Hd = (
        2 * np.pi * exp_params.power_rabi_A_W * 1j * (exp_params.Qd - exp_params.Q) / 2
    )  # 1/2 factor for Ry(pi/2)
    H = [H_dis, [Hd, exp_params.pulse_W]]

    Hdm = (
        -2 * np.pi * exp_params.power_rabi_A_W * 1j * (exp_params.Qd - exp_params.Q) / 2
    )  # 1/2 factor for Ry(pi/2)
    Hm = [H_dis, [Hdm, exp_params.pulse_W]]

    tlist_wait = np.linspace(0, exp_params.twait_W, 11)
    tlist_pi_half = np.linspace(0, exp_params.sigma_W * exp_params.chop_W, 101)

    # first pi/2
    result1 = mesolve(H, rho_init, tlist_pi_half, c_ops=c_ops_qc)

    # dispersive coupling
    result2 = mesolve(H_dis, result1.states[-1], tlist_wait, c_ops=c_ops_qc)

    # second pi/2
    result3 = mesolve(H, result2.states[-1], tlist_pi_half, c_ops=c_ops_qc)

    # second -pi/2
    result3m = mesolve(Hm, result2.states[-1], tlist_pi_half, c_ops=c_ops_qc)

    rho_qt = result3.states[-1].ptrace([0])
    rho_qtm = result3m.states[-1].ptrace([0])

    # w_id[i] = (par * rho_c).tr().real
    pe = (rho_qt * (exp_params.ue * exp_params.ue.dag())).tr()
    pem = (rho_qtm * (exp_params.ue * exp_params.ue.dag())).tr()

    return np.around(pe, 6), np.around(pem, 6)

### Top-level iteration through the T2 values. For each T2 value, iterate through the states, run the protocol, and record in numpy arrays the fidelities, standard deviations, and $\rho_{BME}$ for each. This saves both the normal Wigner and corrected Wigner information. One npz file per T2.

In [7]:
F_MLE_corr = np.zeros(len(state_list), dtype=float)
F_Bayes_BME_corr = np.zeros(len(state_list), dtype=float)

F_MLE_norm = np.zeros(len(state_list), dtype=float)
F_Bayes_BME_norm = np.zeros(len(state_list), dtype=float)

In [8]:
for k, badT2 in enumerate(exp_params.badt2_list):
    print(f"badT2={badT2}")
    for j, state_name in enumerate(state_list):
        if use_grape:
            rho_tar_qc = Y_target(
                state_name, target_states_directory, exp_params.qdim, exp_params.cdim
            )


        else:
            rho_tar_qc = tensor(
                fock_dm(exp_params.qdim, 0),
                ket2dm(cavity_target_state(state_name, exp_params.cdim)),
            )


        Pid_corr = np.zeros(len(exp_params.disp_points))


        Pid_norm = np.zeros(len(exp_params.disp_points))


        if use_grape:

            if state_name in exp_params.pes_after_grape_sim[:, 0]:
                pge = exp_params.pes_after_grape_sim[
                    (exp_params.pes_after_grape_sim[:, 0] == state_name), 1
                ].astype(float)


            else:

                pge = exp_params.pe_after_grape_sim_avg

        else:
            pge = 0


        for w in range(len(exp_params.disp_points)):

            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),

                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),

                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )

            )
            Tphi_badt2 = 1 / (1 / badT2 - 0.5 / exp_params.T1)
            c_ops_qc = [
                # Qubit Relaxation
                np.sqrt((1 + exp_params.nbar_qb) / exp_params.T1) * exp_params.Q,
                # Qubit Thermal Excitations
                np.sqrt(exp_params.nbar_qb / exp_params.T1) * exp_params.Qd,
                # Qubit Dephasing, changed
                np.sqrt(2 / Tphi_badt2) * exp_params.Qd * exp_params.Q,
                # Cavity Relaxation
                np.sqrt((1 + exp_params.nbar_cav) / exp_params.cavT1) * exp_params.C,
                # Cavity Thermal Excitations
                np.sqrt(exp_params.nbar_cav / exp_params.cavT1) * exp_params.Cd,
            ]


            pe, pem = wigner_sim(RHO_tar, c_ops_qc)


            Pid_corr[w] = (
                (pe - pge) / (1 - 2 * pge) - (pem - pge) / (1 - 2 * pge)
            ).squeeze()


            Pid_norm[w] = (2 * (pe - pge) / (1 - 2 * pge) - 1).squeeze()
        ##################################################################


        rho_est_corr, qRho_est_corr = get_LS_and_MLE_rho_est(
            Pid_corr, W, beta, exp_params.D, exp_params.nD
        )


        rho_est_norm, qRho_est_norm = get_LS_and_MLE_rho_est(
            Pid_norm, W, beta, exp_params.D, exp_params.nD
        )


        rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])


        rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()


        Fmean_bayes_corr, Fstd_bayes_corr, rho_BME_corr = bayesian_rho_est(

            numSamp=2**10,
            N=exp_params.N_sim,
            rho_tar=rho_tar_c,
            rhoLS=rho_est_corr.full(),
        )


        Fmean_bayes_norm, Fstd_bayes_norm, rho_BME_norm = bayesian_rho_est(

            numSamp=2**10,
            N=exp_params.N_sim,
            rho_tar=rho_tar_c,
            rhoLS=rho_est_norm.full(),
        )


        F_Bayes_BME_corr[j] = fidelity(rho_tar_c, Qobj(rho_BME_corr)) ** 2

        F_Bayes_BME_norm[j] = fidelity(rho_tar_c, Qobj(rho_BME_norm)) ** 2


        F_MLE_corr[j] = fidelity(rho_tar_c, qRho_est_corr) ** 2

        F_MLE_norm[j] = fidelity(rho_tar_c, qRho_est_norm) ** 2


        print(f"state {j}: {state_name}, corr: {F_Bayes_BME_corr[j]}")

        print(f"state {j}: {state_name}, norm: {F_Bayes_BME_norm[j]}\n")
    if use_grape:
        np.savez(
            f"results_t2\sim\W\grape_target_states\Bayes_F_rrno={k}_sim_W.npz",
            F_MLE_corr=F_MLE_corr,
            F_Bayes_BME_corr=F_Bayes_BME_corr,
            F_MLE_norm=F_MLE_norm,
            F_Bayes_BME_norm=F_Bayes_BME_norm,
        )
    else:
        np.savez(
            f"results_t2\sim\W\ideal_target_states\Bayes_F_rrno={k}_sim_W.npz",
            F_MLE_corr=F_MLE_corr,
            F_Bayes_BME_corr=F_Bayes_BME_corr,
            F_MLE_norm=F_MLE_norm,
            F_Bayes_BME_norm=F_Bayes_BME_norm,
        )

badT2=22389.12
state 0: cat-eve-1, corr: 0.9834220676990738
state 0: cat-eve-1, norm: 0.7416268002495368

